# <span style="color: #007BFF;">Use Note for Camera Acquisition and Display System</span>

This Python script utilizes the PySpin library to interface with multiple cameras, allowing real-time video capture, display, and saving. The system is controlled through a graphical user interface (GUI) created with Tkinter. Here's a brief overview of how to use the script:

## <span style="color: #28A745;">Setup and Operation</span>

### <span style="color: #17A2B8;">Camera Detection and Initialization</span>

- **Automatic Detection:** The script automatically detects all connected cameras and retrieves their serial numbers.
- **Configuration:** Each camera will be configured to start capturing images based on the settings provided by the user in the GUI.

### <span style="color: #17A2B8;">GUI Configuration</span>

The GUI provides fields to set camera parameters:
- **Exposure Time (µs):** Default is 5000 µs.
- **Frame Rate (fps):** Default is 60 fps.
- **Filename Prefix:** Default is 'output'.

**Actions:**
- **Start Acquisition:** Press the "Start Acquisition" button to begin capturing and saving video from all detected cameras.
- **Stop Acquisition:** Press the "Stop Acquisition" button to stop the capture and processing.

### <span style="color: #17A2B8;">Frame Capture and Display</span>

- **Video Capture:** The system captures video frames at the specified frame rate and exposure time.
- **Display:** Each frame is displayed in a separate window at half the original size.

### <span style="color: #dc3545;">Important Note: Square Pulse Synchronization</span>

- **Synchronization:** Ensure that you send a square wave pulse at the frame rate you wish to capture. This pulse should match the frame rate set in the GUI to ensure accurate timing and data capture. This step is crucial for precise synchronization between the camera's frame rate and the data acquisition process.

### <span style="color: #17A2B8;">Saving Videos</span>

- **File Naming:** Video files are saved with filenames that include the current date, time, camera serial number, and the user-defined prefix.
- **Format:** The format used for video saving is AVI, with the XVID codec.

### <span style="color: #17A2B8;">System Requirements</span>

- **Dependencies:** Ensure that the PySpin library and OpenCV are correctly installed and configured in your Python environment.
- **Hardware:** Make sure the cameras are properly connected and powered on.

### <span style="color: #17A2B8;">Important Notes</span>

- **Error Handling:** The script includes error handling for incomplete images and camera initialization issues. Any problems will be printed to the console.
- **Stopping Display:** Press the Escape key to stop the display of frames; however, the video recording will continue until acquisition is explicitly stopped.

### <span style="color: #17A2B8;">Troubleshooting</span>

- **No Cameras Detected:** Verify that the cameras are connected and powered on. Check the connections and try restarting the script.
- **Display Issues:** Ensure OpenCV is installed correctly and there are no issues with GUI rendering.
- **Synchronization Problems:** Check that the pulse signal matches the frame rate specified in the GUI.

This script provides a robust solution for multi-camera video acquisition and analysis, suitable for various research and industrial applications.


In [1]:
import PySpin
import cv2
import threading
import queue
import tkinter as tk
from datetime import datetime

# Default values for camera settings
DEFAULT_EXPOSURE_TIME = 5000  # Exposure time in microseconds
DEFAULT_FRAME_RATE = 60       # Frame rate in frames per second
DEFAULT_FILENAME_PREFIX = 'output'  # Default prefix for output video files

# Global variables for tracking frames and managing threads
frames_written = {}
frames_dropped = {}
frame_queues = {}
display_flags = {}
stop_flag = threading.Event()
threads = []

def get_camera_serial_numbers():
    """
    Retrieve and print serial numbers of all connected cameras.

    Returns:
        list: A list of serial numbers for all connected cameras.
    """
    try:
        system = PySpin.System.GetInstance()
        camera_list = system.GetCameras()
        serial_numbers = []

        for i in range(camera_list.GetSize()):
            camera = camera_list.GetByIndex(i)
            try:
                camera.Init()
                serial_number = PySpin.CStringPtr(camera.GetNodeMap().GetNode('DeviceSerialNumber')).GetValue()
                serial_numbers.append(serial_number)
                camera.DeInit()
            except PySpin.SpinnakerException as ex:
                print(f"Error with camera {i}: {ex}")

        camera_list.Clear()
        system.ReleaseInstance()
        return serial_numbers

    except PySpin.SpinnakerException as ex:
        print(f"An error occurred: {ex}")
        return []

def display_frames(serial, width, height):
    """
    Thread function to display frames from a camera in a separate window at half size.

    Args:
        serial (str): The serial number of the camera.
        width (int): The width of the video frames.
        height (int): The height of the video frames.
    """
    global frame_queues

    # Calculate new dimensions (half the size)
    display_width = width // 2
    display_height = height // 2

    while not stop_flag.is_set() or not frame_queues[serial].empty():
        if not frame_queues[serial].empty():
            frame = frame_queues[serial].get()
            if frame is None:
                break

            # Resize frame to half its original size
            frame_resized = cv2.resize(frame, (display_width, display_height))

            # Display the resized frame
            cv2.imshow(f'Camera {serial}', frame_resized)

            # Check for Escape key press and stop acquisition if pressed
            if cv2.waitKey(1) == 27:  # Escape key pressed
                stop_flag.set()
                break

    cv2.destroyWindow(f'Camera {serial}')



def acquire_images(serial, exposure_time, frame_rate, filename_prefix):
    """
    Thread function to acquire and save images from a camera.

    Args:
        serial (str): The serial number of the camera.
        exposure_time (int): The exposure time in microseconds.
        frame_rate (int): The frame rate in frames per second.
        filename_prefix (str): The prefix for the output video file.
    """
    global frames_written, frames_dropped

    blackFly = None
    try:
        # Retrieve and initialize the camera based on its serial number
        blackFly = PySpin.System.GetInstance().GetCameras().GetBySerial(serial)
        blackFly.Init()
        nodemap = blackFly.GetNodeMap()

        # Configure camera settings
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        node_acquisition_mode.SetIntValue(node_acquisition_mode_continuous.GetValue())

        node_pixel_format = PySpin.CEnumerationPtr(nodemap.GetNode('PixelFormat'))
        node_pixel_format_mono8 = node_pixel_format.GetEntryByName('Mono8')
        node_pixel_format.SetIntValue(node_pixel_format_mono8.GetValue())

        node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
        max_width = node_width.GetMax()
        node_width.SetValue(max_width)
        node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
        max_height = node_height.GetMax()
        node_height.SetValue(max_height)

        node_acquisition_frame_rate_enable = PySpin.CBooleanPtr(nodemap.GetNode('AcquisitionFrameRateEnable'))
        node_acquisition_frame_rate_enable.SetValue(True)
        node_frame_rate = PySpin.CFloatPtr(nodemap.GetNode('AcquisitionFrameRate'))
        frame_rate_max = node_frame_rate.GetMax()
        frame_rate = min(frame_rate, frame_rate_max)
        node_frame_rate.SetValue(frame_rate)

        node_exposure_auto = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureAuto'))
        node_exposure_auto_off = node_exposure_auto.GetEntryByName('Off')
        node_exposure_auto.SetIntValue(node_exposure_auto_off.GetValue())
        node_gain_auto = PySpin.CEnumerationPtr(nodemap.GetNode('GainAuto'))
        node_gain_auto_off = node_gain_auto.GetEntryByName('Off')
        node_gain_auto.SetIntValue(node_gain_auto_off.GetValue())
        node_exposure_time = PySpin.CFloatPtr(nodemap.GetNode('ExposureTime'))
        node_exposure_time.SetValue(exposure_time)
        node_gain = PySpin.CFloatPtr(nodemap.GetNode('Gain'))
        node_gain.SetValue(10)  # Set gain to 10 dB

        node_trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))
        node_trigger_mode_on = node_trigger_mode.GetEntryByName('On')
        node_trigger_mode.SetIntValue(node_trigger_mode_on.GetValue())
        node_trigger_selector = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSelector'))
        node_trigger_selector_frame_start = node_trigger_selector.GetEntryByName('FrameStart')
        node_trigger_selector.SetIntValue(node_trigger_selector_frame_start.GetValue())
        node_trigger_source = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSource'))
        node_trigger_source_line3 = node_trigger_source.GetEntryByName('Line3')
        node_trigger_source.SetIntValue(node_trigger_source_line3.GetValue())
        node_trigger_activation = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerActivation'))
        node_trigger_activation_rising_edge = node_trigger_activation.GetEntryByName('RisingEdge')
        node_trigger_activation.SetIntValue(node_trigger_activation_rising_edge.GetValue())
        node_trigger_overlap = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerOverlap'))
        node_trigger_overlap_read_out = node_trigger_overlap.GetEntryByName('ReadOut')
        node_trigger_overlap.SetIntValue(node_trigger_overlap_read_out.GetValue())
        node_trigger_delay = PySpin.CFloatPtr(nodemap.GetNode('TriggerDelay'))
        node_trigger_delay.SetValue(14)
        node_exposure_mode = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureMode'))
        node_exposure_mode_timed = node_exposure_mode.GetEntryByName('Timed')
        node_exposure_mode.SetIntValue(node_exposure_mode_timed.GetValue())

        # Get the height and width of the frames
        height = blackFly.Height.GetValue()
        width = blackFly.Width.GetValue()
        channels = 1  # Assuming grayscale; change to 3 for color images

        # Create VideoWriter object to save video
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        now = datetime.now()
        date_str = now.strftime("%Y%m%d")
        time_str = now.strftime("%H%M")
        filename = f'{date_str}_{time_str}_{serial}_{filename_prefix}.avi'
        out = cv2.VideoWriter(filename, fourcc, frame_rate, (width, height), False)

        # Start camera acquisition
        blackFly.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous)
        blackFly.BeginAcquisition()

        print(f"Video capture started at {frame_rate} fps for camera {serial}.")

        # Start the frame display thread
        display_thread = threading.Thread(target=display_frames, args=(serial, width, height))
        display_thread.start()

        while not stop_flag.is_set():
            im = blackFly.GetNextImage()

            if im.IsIncomplete():
                print(f"Image incomplete with image status {im.GetImageStatus()} for camera {serial}")
                frames_dropped[serial] += 1
                im.Release()
                continue

            # Convert image to OpenCV format and write to video file
            im_cv2_format = im.GetData().reshape((height, width, channels))
            out.write(im_cv2_format)
            frames_written[serial] += 1

            # Add frame to queue for display
            if frame_queues[serial].qsize() < frame_queues[serial].maxsize:
                frame_queues[serial].put(im_cv2_format)

            im.Release()

        # Signal the display thread to stop and wait for it to finish
        frame_queues[serial].put(None)
        display_thread.join()

        print(f"Video capture complete for camera {serial}. Total frames written: {frames_written[serial]}")
        print(f"Total frames dropped for camera {serial}: {frames_dropped[serial]}")

        # End acquisition and release resources
        blackFly.EndAcquisition()
        blackFly.DeInit()
        out.release()

    except PySpin.SpinnakerException as ex:
        print(f"Error with camera {serial}: {ex}")

    finally:
        if blackFly:
            blackFly.DeInit()

def start_acquisition():
    """
    Start image acquisition for all cameras based on user input from the GUI.
    """
    global threads

    exposure_time = int(exposure_entry.get())
    frame_rate = float(frame_rate_entry.get())
    filename_prefix = filename_entry.get()

    if not filename_prefix:
        filename_prefix = DEFAULT_FILENAME_PREFIX

    for serial in serials:
        frames_written[serial] = 0
        frames_dropped[serial] = 0
        frame_queues[serial] = queue.Queue(maxsize=2)
        display_flags[serial] = threading.Event()
        thread = threading.Thread(target=acquire_images, args=(serial, exposure_time, frame_rate, filename_prefix))
        threads.append(thread)
        thread.start()

def stop_acquisition():
    """
    Stop image acquisition for all cameras and clean up resources.
    """
    global stop_flag, threads

    # Set the stop flag to signal all threads to stop
    stop_flag.set()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    # Clear the thread list and reset stop flag
    threads.clear()
    stop_flag.clear()

    print("Acquisition stopped for all cameras.")


def create_gui():
    """
    Create and manage the GUI for user input and control.
    """
    global exposure_entry, frame_rate_entry, filename_entry

    root = tk.Tk()
    root.title("Camera Settings")

    # Create and place GUI components
    tk.Label(root, text="Exposure Time (µs):").grid(row=0, column=0, padx=10, pady=10)
    exposure_entry = tk.Entry(root)
    exposure_entry.insert(0, str(DEFAULT_EXPOSURE_TIME))
    exposure_entry.grid(row=0, column=1, padx=10, pady=10)

    tk.Label(root, text="Frame Rate (fps):").grid(row=1, column=0, padx=10, pady=10)
    frame_rate_entry = tk.Entry(root)
    frame_rate_entry.insert(0, str(DEFAULT_FRAME_RATE))
    frame_rate_entry.grid(row=1, column=1, padx=10, pady=10)

    tk.Label(root, text="Filename:").grid(row=2, column=0, padx=10, pady=10)
    filename_entry = tk.Entry(root)
    filename_entry.insert(0, DEFAULT_FILENAME_PREFIX)
    filename_entry.grid(row=2, column=1, padx=10, pady=10)

    tk.Button(root, text="Start Acquisition", command=start_acquisition).grid(row=3, column=0, padx=10, pady=10)
    tk.Button(root, text="Stop Acquisition", command=stop_acquisition).grid(row=3, column=1, padx=10, pady=10)

    root.mainloop()

# Retrieve and print serial numbers
serials = get_camera_serial_numbers()

# Initialize the PySpin system and retrieve the list of connected cameras
system = PySpin.System.GetInstance()
blackFly_list = system.GetCameras()

# Start the GUI
create_gui()

# Release system instance and close OpenCV windows
system.ReleaseInstance()
cv2.destroyAllWindows()


Video capture started at 59.95 fps for camera 16290112.Video capture started at 59.95 fps for camera 16290104.

Video capture complete for camera 16290112. Total frames written: 4339
Total frames dropped for camera 16290112: 0
Video capture complete for camera 16290104. Total frames written: 4429
Total frames dropped for camera 16290104: 0
Acquisition stopped for all cameras.


In [2]:
import PySpin
import cv2
import threading
import queue
import tkinter as tk
from datetime import datetime

# Default values for camera settings
DEFAULT_EXPOSURE_TIME = 5000  # Exposure time in microseconds
DEFAULT_FRAME_RATE = 60       # Frame rate in frames per second
DEFAULT_FILENAME_PREFIX = 'output'  # Default prefix for output video files

# Global variables for tracking frames and managing threads
frames_written = {}
frames_dropped = {}
frame_queues = {}
display_flags = {}
stop_flag = threading.Event()
threads = []
desired_width = 1280  # for example
desired_height = 640  # for example


def get_camera_serial_numbers():
    """
    Retrieve and print serial numbers of all connected cameras.

    Returns:
        list: A list of serial numbers for all connected cameras.
    """
    try:
        system = PySpin.System.GetInstance()
        camera_list = system.GetCameras()
        serial_numbers = []

        for i in range(camera_list.GetSize()):
            camera = camera_list.GetByIndex(i)
            try:
                camera.Init()
                serial_number = PySpin.CStringPtr(camera.GetNodeMap().GetNode('DeviceSerialNumber')).GetValue()
                serial_numbers.append(serial_number)
                camera.DeInit()
            except PySpin.SpinnakerException as ex:
                print(f"Error with camera {i}: {ex}")

        camera_list.Clear()
        system.ReleaseInstance()
        return serial_numbers

    except PySpin.SpinnakerException as ex:
        print(f"An error occurred: {ex}")
        return []

def display_frames(serial, width, height):
    """
    Thread function to display frames from a camera in a separate window at half size.

    Args:
        serial (str): The serial number of the camera.
        width (int): The width of the video frames.
        height (int): The height of the video frames.
    """
    global frame_queues

    # Calculate new dimensions (half the size)
    display_width = width // 2
    display_height = height // 2

    while not stop_flag.is_set() or not frame_queues[serial].empty():
        if not frame_queues[serial].empty():
            frame = frame_queues[serial].get()
            if frame is None:
                break

            # Resize frame to half its original size
            frame_resized = cv2.resize(frame, (display_width, display_height))

            # Display the resized frame
            cv2.imshow(f'Camera {serial}', frame_resized)

            # Check for Escape key press and stop acquisition if pressed
            if cv2.waitKey(1) == 27:  # Escape key pressed
                stop_flag.set()
                break

    cv2.destroyWindow(f'Camera {serial}')



def acquire_images(serial, exposure_time, frame_rate, filename_prefix):
    """
    Thread function to acquire and save images from a camera.

    Args:
        serial (str): The serial number of the camera.
        exposure_time (int): The exposure time in microseconds.
        frame_rate (int): The frame rate in frames per second.
        filename_prefix (str): The prefix for the output video file.
    """
    global frames_written, frames_dropped

    blackFly = None
    try:
        # Retrieve and initialize the camera based on its serial number
        blackFly = PySpin.System.GetInstance().GetCameras().GetBySerial(serial)
        blackFly.Init()
        nodemap = blackFly.GetNodeMap()

        # Configure camera settings
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        node_acquisition_mode.SetIntValue(node_acquisition_mode_continuous.GetValue())

        node_pixel_format = PySpin.CEnumerationPtr(nodemap.GetNode('PixelFormat'))
        node_pixel_format_mono8 = node_pixel_format.GetEntryByName('Mono8')
        node_pixel_format.SetIntValue(node_pixel_format_mono8.GetValue())

        node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
        max_width = node_width.GetMax()
        node_width.SetValue(desired_width)
        node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
        max_height = node_height.GetMax()
        node_height.SetValue(desired_height)

        node_acquisition_frame_rate_enable = PySpin.CBooleanPtr(nodemap.GetNode('AcquisitionFrameRateEnable'))
        node_acquisition_frame_rate_enable.SetValue(True)
        node_frame_rate = PySpin.CFloatPtr(nodemap.GetNode('AcquisitionFrameRate'))
        frame_rate_max = node_frame_rate.GetMax()
        frame_rate = min(frame_rate, frame_rate_max)
        node_frame_rate.SetValue(frame_rate)

        node_exposure_auto = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureAuto'))
        node_exposure_auto_off = node_exposure_auto.GetEntryByName('Off')
        node_exposure_auto.SetIntValue(node_exposure_auto_off.GetValue())
        node_gain_auto = PySpin.CEnumerationPtr(nodemap.GetNode('GainAuto'))
        node_gain_auto_off = node_gain_auto.GetEntryByName('Off')
        node_gain_auto.SetIntValue(node_gain_auto_off.GetValue())
        node_exposure_time = PySpin.CFloatPtr(nodemap.GetNode('ExposureTime'))
        node_exposure_time.SetValue(exposure_time)
        node_gain = PySpin.CFloatPtr(nodemap.GetNode('Gain'))
        node_gain.SetValue(10)  # Set gain to 10 dB

        node_trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))
        node_trigger_mode_on = node_trigger_mode.GetEntryByName('On')
        node_trigger_mode.SetIntValue(node_trigger_mode_on.GetValue())
        node_trigger_selector = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSelector'))
        node_trigger_selector_frame_start = node_trigger_selector.GetEntryByName('FrameStart')
        node_trigger_selector.SetIntValue(node_trigger_selector_frame_start.GetValue())
        node_trigger_source = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSource'))
        node_trigger_source_line3 = node_trigger_source.GetEntryByName('Line3')
        node_trigger_source.SetIntValue(node_trigger_source_line3.GetValue())
        node_trigger_activation = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerActivation'))
        node_trigger_activation_rising_edge = node_trigger_activation.GetEntryByName('RisingEdge')
        node_trigger_activation.SetIntValue(node_trigger_activation_rising_edge.GetValue())
        node_trigger_overlap = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerOverlap'))
        node_trigger_overlap_read_out = node_trigger_overlap.GetEntryByName('ReadOut')
        node_trigger_overlap.SetIntValue(node_trigger_overlap_read_out.GetValue())
        node_trigger_delay = PySpin.CFloatPtr(nodemap.GetNode('TriggerDelay'))
        node_trigger_delay.SetValue(14)
        node_exposure_mode = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureMode'))
        node_exposure_mode_timed = node_exposure_mode.GetEntryByName('Timed')
        node_exposure_mode.SetIntValue(node_exposure_mode_timed.GetValue())

        # Get the height and width of the frames
        height = blackFly.Height.GetValue()
        width = blackFly.Width.GetValue()
        channels = 1  # Assuming grayscale; change to 3 for color images

        # Create VideoWriter object to save video
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        now = datetime.now()
        date_str = now.strftime("%Y%m%d")
        time_str = now.strftime("%H%M")
        filename = f'{date_str}_{time_str}_{serial}_{filename_prefix}.avi'
        out = cv2.VideoWriter(filename, fourcc, frame_rate, (width, height), False)

        # Start camera acquisition
        blackFly.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous)
        blackFly.BeginAcquisition()

        print(f"Video capture started at {frame_rate} fps for camera {serial}.")

        # Start the frame display thread
        display_thread = threading.Thread(target=display_frames, args=(serial, width, height))
        display_thread.start()

        while not stop_flag.is_set():
            im = blackFly.GetNextImage()

            if im.IsIncomplete():
                print(f"Image incomplete with image status {im.GetImageStatus()} for camera {serial}")
                frames_dropped[serial] += 1
                im.Release()
                continue

            # Convert image to OpenCV format and write to video file
            im_cv2_format = im.GetData().reshape((height, width, channels))
            out.write(im_cv2_format)
            frames_written[serial] += 1

            # Add frame to queue for display
            if frame_queues[serial].qsize() < frame_queues[serial].maxsize:
                frame_queues[serial].put(im_cv2_format)

            im.Release()

        # Signal the display thread to stop and wait for it to finish
        frame_queues[serial].put(None)
        display_thread.join()

        print(f"Video capture complete for camera {serial}. Total frames written: {frames_written[serial]}")
        print(f"Total frames dropped for camera {serial}: {frames_dropped[serial]}")

        # End acquisition and release resources
        blackFly.EndAcquisition()
        blackFly.DeInit()
        out.release()

    except PySpin.SpinnakerException as ex:
        print(f"Error with camera {serial}: {ex}")

    finally:
        if blackFly:
            blackFly.DeInit()

def start_acquisition():
    """
    Start image acquisition for all cameras based on user input from the GUI.
    """
    global threads

    exposure_time = int(exposure_entry.get())
    frame_rate = float(frame_rate_entry.get())
    filename_prefix = filename_entry.get()

    if not filename_prefix:
        filename_prefix = DEFAULT_FILENAME_PREFIX

    for serial in serials:
        frames_written[serial] = 0
        frames_dropped[serial] = 0
        frame_queues[serial] = queue.Queue(maxsize=2)
        display_flags[serial] = threading.Event()
        thread = threading.Thread(target=acquire_images, args=(serial, exposure_time, frame_rate, filename_prefix))
        threads.append(thread)
        thread.start()

def stop_acquisition():
    """
    Stop image acquisition for all cameras and clean up resources.
    """
    global stop_flag, threads

    # Set the stop flag to signal all threads to stop
    stop_flag.set()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    # Clear the thread list and reset stop flag
    threads.clear()
    stop_flag.clear()

    print("Acquisition stopped for all cameras.")


def create_gui():
    """
    Create and manage the GUI for user input and control.
    """
    global exposure_entry, frame_rate_entry, filename_entry

    root = tk.Tk()
    root.title("Camera Settings")

    # Create and place GUI components
    tk.Label(root, text="Exposure Time (µs):").grid(row=0, column=0, padx=10, pady=10)
    exposure_entry = tk.Entry(root)
    exposure_entry.insert(0, str(DEFAULT_EXPOSURE_TIME))
    exposure_entry.grid(row=0, column=1, padx=10, pady=10)

    tk.Label(root, text="Frame Rate (fps):").grid(row=1, column=0, padx=10, pady=10)
    frame_rate_entry = tk.Entry(root)
    frame_rate_entry.insert(0, str(DEFAULT_FRAME_RATE))
    frame_rate_entry.grid(row=1, column=1, padx=10, pady=10)

    tk.Label(root, text="Filename:").grid(row=2, column=0, padx=10, pady=10)
    filename_entry = tk.Entry(root)
    filename_entry.insert(0, DEFAULT_FILENAME_PREFIX)
    filename_entry.grid(row=2, column=1, padx=10, pady=10)

    tk.Button(root, text="Start Acquisition", command=start_acquisition).grid(row=3, column=0, padx=10, pady=10)
    tk.Button(root, text="Stop Acquisition", command=stop_acquisition).grid(row=3, column=1, padx=10, pady=10)

    root.mainloop()

# Retrieve and print serial numbers
serials = get_camera_serial_numbers()

# Initialize the PySpin system and retrieve the list of connected cameras
system = PySpin.System.GetInstance()
blackFly_list = system.GetCameras()

# Start the GUI
create_gui()

# Release system instance and close OpenCV windows
system.ReleaseInstance()
cv2.destroyAllWindows()


Video capture started at 59.95 fps for camera 16290112.
Video capture started at 59.95 fps for camera 16290104.
Video capture complete for camera 16290112. Total frames written: 449
Total frames dropped for camera 16290112: 0
Video capture complete for camera 16290104. Total frames written: 452
Total frames dropped for camera 16290104: 0
Acquisition stopped for all cameras.
Video capture started at 59.95 fps for camera 16290104.
Video capture started at 59.95 fps for camera 16290112.
Video capture complete for camera 16290104. Total frames written: 651
Total frames dropped for camera 16290104: 0
Video capture complete for camera 16290112. Total frames written: 651
Total frames dropped for camera 16290112: 0
Acquisition stopped for all cameras.
Video capture started at 59.95 fps for camera 16290104.Video capture started at 59.95 fps for camera 16290112.

Video capture complete for camera 16290104. Total frames written: 7371
Total frames dropped for camera 16290104: 0
Video capture compl

KeyboardInterrupt: 